<a href="https://colab.research.google.com/github/FanisBaygildin/Toronto-KSI-Risk-Bot/blob/main/Toronto_KSI_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pygeohash
!pip install astral
!pip install scikit-learn==1.6.1   # for Render compatibility

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 kB 1.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%config IPCompliter.greedy=True
pd.options.display.max_columns = 55

# models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import roc_auc_score, brier_score_loss

from sklearn.impute import KNNImputer

# for date/time
from datetime import datetime, timezone, timedelta
import requests
import datetime as dt
import pytz

from numpy.random import default_rng

# # for squares
import pygeohash as pgh

# # метеоданные
import requests
from astral.sun import sun
from astral import LocationInfo

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from joblib import dump, load

import json

In [ ]:
ksi = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Toronto KSI/Total_KSI.csv')

In [ ]:
ksi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18957 entries, 0 to 18956
Data columns (total 54 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   OBJECTID           18957 non-null  int64  
 1   INDEX              18957 non-null  int64  
 2   ACCNUM             14027 non-null  float64
 3   DATE               18957 non-null  object 
 4   TIME               18957 non-null  int64  
 5   STREET1            18957 non-null  object 
 6   STREET2            17251 non-null  object 
 7   OFFSET             3820 non-null   object 
 8   ROAD_CLASS         18471 non-null  object 
 9   DISTRICT           18728 non-null  object 
 10  LATITUDE           18957 non-null  float64
 11  LONGITUDE          18957 non-null  float64
 12  ACCLOC             13501 non-null  object 
 13  TRAFFCTL           18882 non-null  object 
 14  VISIBILITY         18933 non-null  object 
 15  LIGHT              18953 non-null  object 
 16  RDSFCOND           189

# Cut the table to 2017 - 2020

In [ ]:
ksi[ksi['DATE'].str.contains(r'^ ?1/\d{1}/2017 10:00:00 AM$')].head(3)

,OBJECTID,INDEX,ACCNUM,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,LATITUDE,LONGITUDE,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,x,y
13238,13239,80734609,NaN,1/1/2017 10:00:00 AM,1405,57 BIMINI CRES,NaN,9 m South of,Local,Etobicoke York,43.766555,-79.526157,Non Intersection,No Control,Clear,Daylight,Other,Non-Fatal Injury,SMV Other,Driver,85 to 89,Major,NaN,West,"Automobile, Station Wagon",Reversing,Other,Inattentive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,Black Creek,24,Black Creek (24),D31,618627.2421,4.847000e+06
13239,13240,80890115,NaN,1/3/2017 10:00:00 AM,10,F G GARDINER XY E,KIPLING AVE,900 m West o,NaN,Etobicoke York,43.615851,-79.535251,Non Intersection,No Control,Rain,"Dark, artificial",Wet,Non-Fatal Injury,Rear End,Passenger,20 to 24,Major,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,Yes,Yes,NaN,NaN,NaN,20,Alderwood,20,Alderwood (20),D22,618190.9169,4.830249e+06
13240,13241,80890116,NaN,1/3/2017 10:00:00 AM,10,F G GARDINER XY E,KIPLING AVE,900 m West o,NaN,Etobicoke York,43.615851,-79.535251,Non Intersection,No Control,Rain,"Dark, artificial",Wet,Non-Fatal Injury,Rear End,Passenger,15 to 19,Major,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,Yes,Yes,NaN,NaN,NaN,20,Alderwood,20,Alderwood (20),D22,618190.9169,4.830249e+06


In [ ]:
ksi[ksi['DATE'].str.contains(r'^ ?12/\d{2}/2020 10:00:00 AM$')].tail(3)

,OBJECTID,INDEX,ACCNUM,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,LATITUDE,LONGITUDE,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,x,y
16871,16872,81509301,2449891.0,12/30/2020 10:00:00 AM,1700,DUNDAS ST E,OSKENONTON LANE,54 m East of,Major Arterial,Toronto and East York,43.658408,-79.370212,Non Intersection,No Control,Rain,"Dark, artificial",Wet,Non-Fatal Injury,Pedestrian Collisions,Pedestrian,70 to 74,Major,NaN,South,NaN,NaN,NaN,NaN,Pedestrian hit at mid-block,"Crossing, no Traffic Control",Inattentive,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73,Moss Park,73,Moss Park (73),D51,631415.2812,4.835224e+06
16872,16873,81432824,2450003.0,12/30/2020 10:00:00 AM,1721,KENNEDY RD,SHEPPARD AVE E,NaN,Major Arterial,Scarborough,43.783213,-79.288211,Intersection Related,Traffic Signal,Rain,"Dark, artificial",Wet,Fatal,Pedestrian Collisions,Driver,35 to 39,NaN,NaN,North,"Automobile, Station Wagon",Going Ahead,Other,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128,Agincourt South-Malvern West,128,Agincourt South-Malvern West (128),D42,637741.0669,4.849218e+06
16873,16874,81432825,2450003.0,12/30/2020 10:00:00 AM,1721,KENNEDY RD,SHEPPARD AVE E,NaN,Major Arterial,Scarborough,43.783213,-79.288211,Intersection Related,Traffic Signal,Rain,"Dark, artificial",Wet,Fatal,Pedestrian Collisions,Pedestrian,55 to 59,Fatal,40.0,Unknown,NaN,NaN,NaN,NaN,Vehicle is going straight thru inter.while ped...,Crossing marked crosswalk without ROW,Unknown,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128,Agincourt South-Malvern West,128,Agincourt South-Malvern West (128),D42,637741.0669,4.849218e+06


In [ ]:
ksi = ksi.iloc[13238:16874].reset_index(drop=True)

Checking Unique Values and Percentage of NaN

In [ ]:
for col in ksi.columns:
    print(col)
    print(ksi[col].unique())
    print(ksi[col].isna().sum()/189.57)

OBJECTID
[13239 13240 13241 ... 16872 16873 16874]
0.0
INDEX
[80734609 80890115 80890116 ... 81509301 81432824 81432825]
0.0
ACCNUM
[           nan 1.78057130e+08 1.88016123e+08 2.53010000e+04
 2.62940000e+04 3.73300000e+04 4.56640000e+04 5.68150000e+04
 6.68930000e+04 7.31530000e+04 7.42930000e+04 7.52720000e+04
 7.63040000e+04 9.32490000e+04 9.50010000e+04 1.05298000e+05
 1.17822000e+05 1.18659000e+05 1.21282000e+05 1.33203000e+05
 1.36270000e+05 1.39578000e+05 1.44397000e+05 1.69709000e+05
 1.80484000e+05 1.87607000e+05 1.88526000e+05 2.02070000e+05
 2.12688000e+05 2.13617000e+05 2.14205000e+05 2.20239000e+05
 2.20521000e+05 2.31164000e+05 2.45690000e+05 2.70261000e+05
 2.96685000e+05 3.16011000e+05 3.32841000e+05 3.42262000e+05
 3.48999000e+05 3.74527000e+05 3.82209000e+05 3.85373000e+05
 4.01654000e+05 4.18177000e+05 4.18513000e+05 4.29600000e+05
 4.37574000e+05 4.42335000e+05 4.44225000e+05 4.50742000e+05
 4.50834000e+05 4.51712000e+05 4.74338000e+05 4.84297000e+05
 4.88737000e+0

In [ ]:
ksi.drop(columns = ['OBJECTID', 'INDEX', 'DISTRICT', 'OFFSET', 'ROAD_CLASS', 'ACCLOC', 'STREET1', 'STREET2', 'INVAGE', 'INVTYPE', 'VEHTYPE', 'NEIGHBOURHOOD_158', 'NEIGHBOURHOOD_140', 'x', 'y', 'ACCLASS',  \
                    'FATAL_NO', 'MANOEUVER', 'DRIVACT', 'DRIVCOND', 'PEDTYPE', 'PEDACT', 'PEDCOND', 'CYCLISTYPE', 'CYCACT', 'CYCCOND', 'AUTOMOBILE', 'IMPACTYPE', 'HOOD_158', 'HOOD_140', 'INITDIR', 'TRAFFCTL', 'VISIBILITY', 'LIGHT', 'RDSFCOND',\
                    'PEDESTRIAN', 'CYCLIST',  'MOTORCYCLE', 'TRUCK', 'TRSN_CITY_VEH', 'EMERG_VEH', 'PASSENGER', 'SPEEDING', 'AG_DRIV', 'REDLIGHT', 'ALCOHOL', 'DISABILITY', 'DIVISION', 'INJURY'], inplace=True)

In [ ]:
ksi.head(10)

,ACCNUM,DATE,TIME,LATITUDE,LONGITUDE
0,NaN,1/1/2017 10:00:00 AM,1405,43.766555,-79.526157
1,NaN,1/3/2017 10:00:00 AM,10,43.615851,-79.535251
2,NaN,1/3/2017 10:00:00 AM,10,43.615851,-79.535251
3,NaN,1/3/2017 10:00:00 AM,10,43.615851,-79.535251
4,NaN,1/3/2017 10:00:00 AM,10,43.615851,-79.535251
5,NaN,1/3/2017 10:00:00 AM,10,43.615851,-79.535251
6,NaN,1/3/2017 10:00:00 AM,10,43.615851,-79.535251
7,NaN,1/3/2017 10:00:00 AM,10,43.615851,-79.535251
8,NaN,1/3/2017 10:00:00 AM,717,43.648974,-79.367259
9,NaN,1/3/2017 10:00:00 AM,717,43.648974,-79.367259


In [ ]:
ksi.info()    # only accnum has nan

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3636 entries, 0 to 3635
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ACCNUM     645 non-null    float64
 1   DATE       3636 non-null   object 
 2   TIME       3636 non-null   int64  
 3   LATITUDE   3636 non-null   float64
 4   LONGITUDE  3636 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 142.2+ KB


# Filling ACCNUM NaN

In [ ]:
# ACCNUM is a unique number of in accident, we need it to clean the data and keep 1 row per accident only

ksi['ACCNUM_Filled'] = ksi['ACCNUM']

change = (ksi[['DATE', 'TIME', 'LATITUDE', 'LONGITUDE']] != ksi[['DATE', 'TIME', 'LATITUDE', 'LONGITUDE']].shift()).any(axis=1)         # compare each row with the previous one to create a boolean mask: True: the raws aren't equal, False: the raws are equal.

ksi['ACCNUM_Filled'] = change.cumsum() + 10000     # if current change is True we get an additional 1 to ACCNUM_Filled value

ksi

,ACCNUM,DATE,TIME,LATITUDE,LONGITUDE,ACCNUM_Filled
0,NaN,1/1/2017 10:00:00 AM,1405,43.766555,-79.526157,10001
1,NaN,1/3/2017 10:00:00 AM,10,43.615851,-79.535251,10002
2,NaN,1/3/2017 10:00:00 AM,10,43.615851,-79.535251,10002
3,NaN,1/3/2017 10:00:00 AM,10,43.615851,-79.535251,10002
4,NaN,1/3/2017 10:00:00 AM,10,43.615851,-79.535251,10002
...,...,...,...,...,...,...
3631,2431394.0,12/27/2020 10:00:00 AM,1520,43.748946,-79.277301,11449
3632,2449891.0,12/30/2020 10:00:00 AM,1700,43.658408,-79.370212,11450
3633,2449891.0,12/30/2020 10:00:00 AM,1700,43.658408,-79.370212,11450
3634,2450003.0,12/30/2020 10:00:00 AM,1721,43.783213,-79.288211,11451


In [ ]:
ksi.drop('ACCNUM', axis=1, inplace=True)     # original ACCNUM can be removed

In [ ]:
ksi = ksi.drop_duplicates(subset='ACCNUM_Filled')   # Keeping 1 row per accident
ksi

,DATE,TIME,LATITUDE,LONGITUDE,ACCNUM_Filled
0,1/1/2017 10:00:00 AM,1405,43.766555,-79.526157,10001
1,1/3/2017 10:00:00 AM,10,43.615851,-79.535251,10002
8,1/3/2017 10:00:00 AM,717,43.648974,-79.367259,10003
10,1/7/2017 10:00:00 AM,16,43.787211,-79.470042,10004
13,1/7/2017 10:00:00 AM,2222,43.645438,-79.523776,10005
...,...,...,...,...,...
3624,12/21/2020 10:00:00 AM,2035,43.669074,-79.378078,11447
3626,12/25/2020 10:00:00 AM,318,43.641318,-79.579822,11448
3628,12/27/2020 10:00:00 AM,1520,43.748946,-79.277301,11449
3632,12/30/2020 10:00:00 AM,1700,43.658408,-79.370212,11450


# Geohash5

In [ ]:
def encode5(row):
    return pgh.encode(row['LATITUDE'], row['LONGITUDE'], precision=5)   # func to code a point to geohash

ksi['geohash5'] = ksi.apply(encode5, axis=1)    # понять, как это работает

/tmp/ipython-input-15-4015343631.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ksi['geohash5'] = ksi.apply(encode5, axis=1)    # понять, как это работает


In [ ]:
ksi

,DATE,TIME,LATITUDE,LONGITUDE,ACCNUM_Filled,geohash5
0,1/1/2017 10:00:00 AM,1405,43.766555,-79.526157,10001,dpz2y
1,1/3/2017 10:00:00 AM,10,43.615851,-79.535251,10002,dpz2n
8,1/3/2017 10:00:00 AM,717,43.648974,-79.367259,10003,dpz83
10,1/7/2017 10:00:00 AM,16,43.787211,-79.470042,10004,dpz3p
13,1/7/2017 10:00:00 AM,2222,43.645438,-79.523776,10005,dpz2q
...,...,...,...,...,...,...
3624,12/21/2020 10:00:00 AM,2035,43.669074,-79.378078,11447,dpz83
3626,12/25/2020 10:00:00 AM,318,43.641318,-79.579822,11448,dpz2m
3628,12/27/2020 10:00:00 AM,1520,43.748946,-79.277301,11449,dpz8u
3632,12/30/2020 10:00:00 AM,1700,43.658408,-79.370212,11450,dpz83


In [ ]:
# Преобразуем колонку DATE в datetime, если это ещё не сделано
ksi['DATE'] = pd.to_datetime(ksi['DATE'])

# Извлекаем год, месяц и день
ksi['Year'] = ksi['DATE'].dt.year
ksi['Month'] = ksi['DATE'].dt.month
ksi['Day'] = ksi['DATE'].dt.day

# Округляем TIME до часа
ksi['Hour'] = (ksi['TIME'] // 100).astype(int)
ksi

/tmp/ipython-input-17-1654522438.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ksi['DATE'] = pd.to_datetime(ksi['DATE'])
/tmp/ipython-input-17-1654522438.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ksi['Year'] = ksi['DATE'].dt.year
/tmp/ipython-input-17-1654522438.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

,DATE,TIME,LATITUDE,LONGITUDE,ACCNUM_Filled,geohash5,Year,Month,Day,Hour
0,2017-01-01 10:00:00,1405,43.766555,-79.526157,10001,dpz2y,2017,1,1,14
1,2017-01-03 10:00:00,10,43.615851,-79.535251,10002,dpz2n,2017,1,3,0
8,2017-01-03 10:00:00,717,43.648974,-79.367259,10003,dpz83,2017,1,3,7
10,2017-01-07 10:00:00,16,43.787211,-79.470042,10004,dpz3p,2017,1,7,0
13,2017-01-07 10:00:00,2222,43.645438,-79.523776,10005,dpz2q,2017,1,7,22
...,...,...,...,...,...,...,...,...,...,...
3624,2020-12-21 10:00:00,2035,43.669074,-79.378078,11447,dpz83,2020,12,21,20
3626,2020-12-25 10:00:00,318,43.641318,-79.579822,11448,dpz2m,2020,12,25,3
3628,2020-12-27 10:00:00,1520,43.748946,-79.277301,11449,dpz8u,2020,12,27,15
3632,2020-12-30 10:00:00,1700,43.658408,-79.370212,11450,dpz83,2020,12,30,17


In [ ]:
# making a slot for each row
ksi['slot'] = pd.to_datetime({
    'year':  ksi['Year'],
    'month': ksi['Month'],
    'day':   ksi['Day'],
    'hour':  ksi['Hour']
})

/tmp/ipython-input-18-876034065.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ksi['slot'] = pd.to_datetime({


In [ ]:
ksi['target'] = 1

/tmp/ipython-input-19-1234829135.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ksi['target'] = 1


In [ ]:
# calculating how many geohash squares we get with precision 5

min_lat, max_lat = 43.58, 43.85
min_lon, max_lon = -79.64, -79.12
step = 0.003                     # ≈ 0.45 км should be fine for precision 6

for precision in range(4,7):
    geos = set()
    lat_grid = np.arange(min_lat, max_lat, step)
    lon_grid = np.arange(min_lon, max_lon, step)

    for lat in lat_grid:
        for lon in lon_grid:
            geos.add(pgh.encode(lat, lon, precision=precision))

    all_geohash = sorted(geos)     # готовый список 91 squares
    print(len(all_geohash))        # проверка

6
91
2450


I'll take precision 5, as precision 6 will give us too many rows

In [ ]:
geos = set()
lat_grid = np.arange(min_lat, max_lat, step)
lon_grid = np.arange(min_lon, max_lon, step)

for lat in lat_grid:
    for lon in lon_grid:
        geos.add(pgh.encode(lat, lon, precision=5))

all_geohash5 = sorted(geos)     # готовый список 91 squares
print(len(all_geohash5))        # проверка

91


# Making all combinations of slots for each hour in range 2017-2020 and each geohash5 of Toronto to concatenate with Weather 2017-2020 DF

In [ ]:
slots  = pd.date_range(start='2017-01-01 00:00:00', end='2020-12-31 23:00:00', freq='1h')
len(slots)

35064

In [ ]:
grid = pd.MultiIndex.from_product([slots, all_geohash5], names=['slot','geohash5']).to_frame(index=False)
grid

,slot,geohash5
0,2017-01-01 00:00:00,dpxrg
1,2017-01-01 00:00:00,dpxru
2,2017-01-01 00:00:00,dpxrv
3,2017-01-01 00:00:00,dpxry
4,2017-01-01 00:00:00,dpxrz
...,...,...
3190819,2020-12-31 23:00:00,dpz9m
3190820,2020-12-31 23:00:00,dpz9n
3190821,2020-12-31 23:00:00,dpz9p
3190822,2020-12-31 23:00:00,dpz9q


# Weather 2017-2020 preprocessing

In [ ]:
# Concat .csv filse for each month within 2017-2020

# weather = pd.DataFrame()
# for j in range(2017, 2021):
#     for i in range(1, 13):
#         temp_df = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/Toronto KSI/Toronto Weather 2017-2020/Weather {j}/{j} ({i}).csv')
#         weather = pd.concat([weather, temp_df], ignore_index=True)

# weather.to_csv('/content/drive/MyDrive/Colab Notebooks/Toronto KSI/Toronto Weather 2017-2020/complete_weather.csv')

In [ ]:
weather = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Toronto KSI/Toronto Weather 2017-2020/complete_weather.csv')

/tmp/ipython-input-25-1143783926.py:1: DtypeWarning: Columns (11,13,15,19,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  weather = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Toronto KSI/Toronto Weather 2017-2020/complete_weather.csv')


In [ ]:
weather

,Unnamed: 0,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Time,Temp (°C),Temp Flag,Dew Point Temp (°C),Dew Point Temp Flag,Rel Hum (%),Rel Hum Flag,Wind Dir (10s deg),Wind Dir Flag,Wind Spd (km/h),Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
0,0,-79.4,43.63,TORONTO CITY CENTRE,6158359,2017-01-01 00:00,2017,1,1,00:00,1.5,NaN,-3.6,NaN,69.0,NaN,26.0,NaN,39.0,NaN,16.1,NaN,99.81,NaN,NaN,NaN,NaN,NaN,NaN
1,1,-79.4,43.63,TORONTO CITY CENTRE,6158359,2017-01-01 01:00,2017,1,1,01:00,1.5,NaN,-3.9,NaN,67.0,NaN,27.0,NaN,35.0,NaN,16.1,NaN,100.01,NaN,NaN,NaN,NaN,NaN,NaN
2,2,-79.4,43.63,TORONTO CITY CENTRE,6158359,2017-01-01 02:00,2017,1,1,02:00,1.0,NaN,-4.3,NaN,68.0,NaN,26.0,NaN,32.0,NaN,16.1,NaN,100.14,NaN,NaN,NaN,NaN,NaN,NaN
3,3,-79.4,43.63,TORONTO CITY CENTRE,6158359,2017-01-01 03:00,2017,1,1,03:00,1.2,NaN,-4.3,NaN,67.0,NaN,26.0,NaN,37.0,NaN,16.1,NaN,100.32,NaN,NaN,NaN,NaN,NaN,NaN
4,4,-79.4,43.63,TORONTO CITY CENTRE,6158359,2017-01-01 04:00,2017,1,1,04:00,1.3,NaN,-4.4,NaN,66.0,NaN,26.0,NaN,28.0,NaN,16.1,NaN,100.48,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,35059,-79.4,43.63,TORONTO CITY CENTRE,6158359,2020-12-31 19:00,2020,12,31,19:00,0.1,NaN,-3.3,NaN,78.0,NaN,27.0,NaN,13.0,NaN,16.1,NaN,101.86,NaN,NaN,NaN,NaN,NaN,NaN
35060,35060,-79.4,43.63,TORONTO CITY CENTRE,6158359,2020-12-31 20:00,2020,12,31,20:00,0.0,NaN,-4.0,NaN,74.0,NaN,30.0,NaN,11.0,NaN,16.1,NaN,101.96,NaN,NaN,NaN,-4.0,NaN,NaN
35061,35061,-79.4,43.63,TORONTO CITY CENTRE,6158359,2020-12-31 21:00,2020,12,31,21:00,0.4,NaN,-4.5,NaN,70.0,NaN,26.0,NaN,8.0,NaN,16.1,NaN,102.02,NaN,NaN,NaN,NaN,NaN,NaN
35062,35062,-79.4,43.63,TORONTO CITY CENTRE,6158359,2020-12-31 22:00,2020,12,31,22:00,-0.1,NaN,-4.9,NaN,70.0,NaN,30.0,NaN,11.0,NaN,16.1,NaN,102.13,NaN,NaN,NaN,-4.0,NaN,NaN


In [ ]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           35064 non-null  int64  
 1   Longitude (x)        35064 non-null  float64
 2   Latitude (y)         35064 non-null  float64
 3   Station Name         35064 non-null  object 
 4   Climate ID           35064 non-null  int64  
 5   Date/Time            35064 non-null  object 
 6   Year                 35064 non-null  int64  
 7   Month                35064 non-null  int64  
 8   Day                  35064 non-null  int64  
 9   Time                 35064 non-null  object 
 10  Temp (°C)            34941 non-null  float64
 11  Temp Flag            89 non-null     object 
 12  Dew Point Temp (°C)  34892 non-null  float64
 13  Dew Point Temp Flag  139 non-null    object 
 14  Rel Hum (%)          34900 non-null  float64
 15  Rel Hum Flag         131 non-null   

In [ ]:
weather['Weather'].unique()

array([nan, 'Fog', 'Rain,Fog', 'Rain', 'Snow', 'Moderate Rain',
       'Moderate Rain,Fog', 'Haze', 'Moderate Snow', 'Heavy Rain,Fog',
       'Thunderstorms,Rain,Fog', 'Thunderstorms,Rain', 'Thunderstorms',
       'Thunderstorms,Moderate Rain', 'Thunderstorms,Moderate Rain,Fog',
       'Thunderstorms,Heavy Rain,Fog', 'Rain,Snow', 'Freezing Rain,Fog',
       'Freezing Rain,Snow', 'Freezing Rain', 'Snow,Blowing Snow',
       'Heavy Snow', 'Haze,Blowing Snow', 'Thunderstorms,Fog',
       'Thunderstorms,Heavy Rain'], dtype=object)

In [ ]:
weather['Date/Time'] = pd.to_datetime(weather['Date/Time'])
weather['Date'] = weather['Date/Time'].dt.normalize()

In [ ]:
grp = weather.groupby('Date').agg(total=('Weather', 'sum'))
print(grp.shape[0])
print(grp[grp['total'] == 0])

# 570 rows are NaN out of 1461 rows, not informative feature

1461
           total
Date            
2017-01-01     0
2017-01-06     0
2017-01-15     0
2017-01-16     0
2017-01-30     0
...          ...
2020-12-06     0
2020-12-07     0
2020-12-08     0
2020-12-13     0
2020-12-18     0

[570 rows x 1 columns]


In [ ]:
weather['slot'] = weather['Date/Time'].dt.floor('h')
weather['Hour'] = pd.to_datetime(weather['Time'], format='%H:%M').dt.hour

weather.drop(columns = ['Unnamed: 0',
                        'Longitude (x)',
                        'Latitude (y)',
                        'Time',
                        'Station Name',
                        'Climate ID',
                        'Wind Dir (10s deg)',
                        'Temp Flag',
                        'Dew Point Temp Flag',
                        'Rel Hum Flag',
                        'Wind Dir Flag',
                        'Wind Spd Flag',
                        'Visibility Flag',
                        'Stn Press Flag',
                        'Hmdx',
                        'Hmdx Flag',
                        'Wind Chill',
                        'Wind Chill Flag',
                        'Weather'     # not enough data to use for Snow/Rain
                        ],
             inplace = True)

weather['Stn Press (kPa)'] = weather['Stn Press (kPa)']*10

weather.rename(columns = {'Temp (°C)': 'temp_c',
                        'Dew Point Temp (°C)': 'dewpoint_c',
                        'Rel Hum (%)': 'humidity',
                        'Wind Spd (km/h)': 'wind_kph',
                        'Visibility (km)': 'vis_km',
                        'Stn Press (kPa)': 'pressure_mb'
                        },
               inplace=True)
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date/Time    35064 non-null  datetime64[ns]
 1   Year         35064 non-null  int64         
 2   Month        35064 non-null  int64         
 3   Day          35064 non-null  int64         
 4   temp_c       34941 non-null  float64       
 5   dewpoint_c   34892 non-null  float64       
 6   humidity     34900 non-null  float64       
 7   wind_kph     35016 non-null  float64       
 8   vis_km       35009 non-null  float64       
 9   pressure_mb  34938 non-null  float64       
 10  Date         35064 non-null  datetime64[ns]
 11  slot         35064 non-null  datetime64[ns]
 12  Hour         35064 non-null  int32         
dtypes: datetime64[ns](3), float64(6), int32(1), int64(3)
memory usage: 3.3 MB


# Weather KNN Imputer

In [ ]:
imputation_features = weather.select_dtypes(exclude='datetime64[ns]').columns
imputation_features

Index(['Year', 'Month', 'Day', 'temp_c', 'dewpoint_c', 'humidity', 'wind_kph',
       'vis_km', 'pressure_mb', 'Hour'],
      dtype='object')

In [ ]:
imputer = KNNImputer(n_neighbors=5, weights='distance')
weather[imputation_features] = imputer.fit_transform(weather[imputation_features])

In [ ]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date/Time    35064 non-null  datetime64[ns]
 1   Year         35064 non-null  float64       
 2   Month        35064 non-null  float64       
 3   Day          35064 non-null  float64       
 4   temp_c       35064 non-null  float64       
 5   dewpoint_c   35064 non-null  float64       
 6   humidity     35064 non-null  float64       
 7   wind_kph     35064 non-null  float64       
 8   vis_km       35064 non-null  float64       
 9   pressure_mb  35064 non-null  float64       
 10  Date         35064 non-null  datetime64[ns]
 11  slot         35064 non-null  datetime64[ns]
 12  Hour         35064 non-null  float64       
dtypes: datetime64[ns](3), float64(10)
memory usage: 3.5 MB


In [ ]:
weather

,Date/Time,Year,Month,Day,temp_c,dewpoint_c,humidity,wind_kph,vis_km,pressure_mb,Date,slot,Hour
0,2017-01-01 00:00:00,2017.0,1.0,1.0,1.5,-3.6,69.0,39.0,16.1,998.1,2017-01-01,2017-01-01 00:00:00,0.0
1,2017-01-01 01:00:00,2017.0,1.0,1.0,1.5,-3.9,67.0,35.0,16.1,1000.1,2017-01-01,2017-01-01 01:00:00,1.0
2,2017-01-01 02:00:00,2017.0,1.0,1.0,1.0,-4.3,68.0,32.0,16.1,1001.4,2017-01-01,2017-01-01 02:00:00,2.0
3,2017-01-01 03:00:00,2017.0,1.0,1.0,1.2,-4.3,67.0,37.0,16.1,1003.2,2017-01-01,2017-01-01 03:00:00,3.0
4,2017-01-01 04:00:00,2017.0,1.0,1.0,1.3,-4.4,66.0,28.0,16.1,1004.8,2017-01-01,2017-01-01 04:00:00,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2020-12-31 19:00:00,2020.0,12.0,31.0,0.1,-3.3,78.0,13.0,16.1,1018.6,2020-12-31,2020-12-31 19:00:00,19.0
35060,2020-12-31 20:00:00,2020.0,12.0,31.0,0.0,-4.0,74.0,11.0,16.1,1019.6,2020-12-31,2020-12-31 20:00:00,20.0
35061,2020-12-31 21:00:00,2020.0,12.0,31.0,0.4,-4.5,70.0,8.0,16.1,1020.2,2020-12-31,2020-12-31 21:00:00,21.0
35062,2020-12-31 22:00:00,2020.0,12.0,31.0,-0.1,-4.9,70.0,11.0,16.1,1021.3,2020-12-31,2020-12-31 22:00:00,22.0


In [ ]:
unique_slots = grid['slot']
grid2 = pd.MultiIndex.from_product([slots, all_geohash5], names=['slot', 'geohash5']).to_frame(index=False)
full_ksi_df = grid2.merge(weather, on='slot', how='left')
full_ksi_df['target'] = 0

In [ ]:
full_ksi_df.drop('Year', axis=1, inplace=True)

In [ ]:
full_ksi_df[['Month', 'Day', 'Hour']] = full_ksi_df[['Month', 'Day', 'Hour']].astype('int')  # as KNN converted them to float

In [ ]:
full_ksi_df.isna().sum()

,0
slot,0
geohash5,0
Date/Time,0
Month,0
Day,0
temp_c,0
dewpoint_c,0
humidity,0
wind_kph,0
vis_km,0


In [ ]:
# Создаем множество уникальных пар (slot, geohash5) из KSI
ksi_pairs = set(zip(ksi['slot'], ksi['geohash5']))

# Помечаем target = 1, если пара (slot, geohash5) есть в ksi
full_ksi_df['target'] = full_ksi_df.apply(
    lambda row: 1 if (row['slot'], row['geohash5']) in ksi_pairs else row['target'], axis=1
)

In [ ]:
full_ksi_df[full_ksi_df['target'] == 1]

,slot,geohash5,Date/Time,Month,Day,temp_c,dewpoint_c,humidity,wind_kph,vis_km,pressure_mb,Date,Hour,target
1305,2017-01-01 14:00:00,dpz2y,2017-01-01 14:00:00,1,1,3.0,-5.5,54.0,17.0,16.1,1012.1,2017-01-01,14,1
4389,2017-01-03 00:00:00,dpz2n,2017-01-03 00:00:00,1,3,3.8,2.9,94.0,18.0,11.3,1008.8,2017-01-03,0,1
5051,2017-01-03 07:00:00,dpz83,2017-01-03 07:00:00,1,3,4.3,4.0,98.0,17.0,4.8,1002.4,2017-01-03,7,1
13144,2017-01-07 00:00:00,dpz3p,2017-01-07 00:00:00,1,7,-10.5,-15.5,67.0,15.0,16.1,1016.6,2017-01-07,0,1
15129,2017-01-07 22:00:00,dpz2q,2017-01-07 22:00:00,1,7,-10.1,-14.6,70.0,34.0,16.1,1015.6,2017-01-07,22,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3168666,2020-12-21 20:00:00,dpz83,2020-12-21 20:00:00,12,21,3.4,1.9,90.0,13.0,8.1,995.3,2020-12-21,20,1
3175829,2020-12-25 03:00:00,dpz2m,2020-12-25 03:00:00,12,25,-1.3,-2.9,89.0,28.0,3.6,988.6,2020-12-25,3,1
3181338,2020-12-27 15:00:00,dpz8u,2020-12-27 15:00:00,12,27,0.9,-3.9,71.0,13.0,16.1,1008.9,2020-12-27,15,1
3188049,2020-12-30 17:00:00,dpz83,2020-12-30 17:00:00,12,30,4.5,3.8,95.0,28.0,11.3,998.6,2020-12-30,17,1


In [ ]:
full_ksi_df['Latitude'], full_ksi_df['Longitude'] = zip(*full_ksi_df['geohash5'].apply(pgh.decode))

In [ ]:
full_ksi_df.drop(columns=['slot', 'Date/Time', 'Date', 'geohash5'], inplace=True)

In [ ]:
full_ksi_df

,Month,Day,temp_c,dewpoint_c,humidity,wind_kph,vis_km,pressure_mb,Hour,target,Latitude,Longitude
0,1,1,1.5,-3.6,69.0,39.0,16.1,998.1,0,0,43.571777,-79.650879
1,1,1,1.5,-3.6,69.0,39.0,16.1,998.1,0,0,43.571777,-79.606934
2,1,1,1.5,-3.6,69.0,39.0,16.1,998.1,0,0,43.571777,-79.562988
3,1,1,1.5,-3.6,69.0,39.0,16.1,998.1,0,0,43.571777,-79.519043
4,1,1,1.5,-3.6,69.0,39.0,16.1,998.1,0,0,43.571777,-79.475098
...,...,...,...,...,...,...,...,...,...,...,...,...
3190819,12,31,-1.0,-4.9,75.0,4.0,16.1,1021.3,23,0,43.835449,-79.211426
3190820,12,31,-1.0,-4.9,75.0,4.0,16.1,1021.3,23,0,43.791504,-79.167480
3190821,12,31,-1.0,-4.9,75.0,4.0,16.1,1021.3,23,0,43.791504,-79.123535
3190822,12,31,-1.0,-4.9,75.0,4.0,16.1,1021.3,23,0,43.835449,-79.167480


Список колонок

In [ ]:
columns = full_ksi_df.drop(columns=['target', 'Latitude', 'Longitude']).columns.tolist()
path = '/content/drive/MyDrive/Colab Notebooks/Toronto KSI/columns.json'
with open(path, 'w') as f:
    json.dump(columns, f)

Split

In [ ]:
full_ksi_df['target'].value_counts(normalize=True)   # checking disbalance

,proportion
target,
0,0.999547
1,0.000453


In [ ]:
X = full_ksi_df.drop('target', axis=1)
y = full_ksi_df['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
y_test.value_counts(normalize=True)

,proportion
target,
0,0.999511
1,0.000489


In [ ]:
# models = {
#     "LogisticRegression": LogisticRegression(max_iter=1000),
#     "RandomForest": RandomForestClassifier(n_estimators=100),
#     "GradientBoosting": GradientBoostingClassifier(),
#     "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss"),
#     "LightGBM": LGBMClassifier()
# }

In [ ]:
# from sklearn.metrics import roc_auc_score, brier_score_loss

# for name, model in models.items():
#     model.fit(X_train, y_train)

#     proba = model.predict_proba(X_test)[:, 1]  # вероятность положительного класса

#     auc = roc_auc_score(y_test, proba)
#     brier = brier_score_loss(y_test, proba)

#     print(f"{name}: AUC = {auc:.4f}, Brier Score = {brier:.4f}")

In [ ]:
import time

rfc = RandomForestClassifier(n_estimators=150, max_depth=7, class_weight='balanced', n_jobs=-1)

start = time.time()
rfc.fit(X_train, y_train)
print(f"{(time.time() - start)/60:.2f} minutes")

proba = rfc.predict_proba(X_test)[:, 1]  # вероятность положительного класса

auc = roc_auc_score(y_test, proba)

print(f"AUC = {auc:.4f}")

9.45 minutes
AUC = 0.8101


In [ ]:
class_counts = y_train.value_counts()
scale = class_counts[0] / class_counts[1]

XGB = XGBClassifier(
    n_estimators=6000,
    max_depth=2,
    learning_rate=0.01,
    scale_pos_weight=scale,  # учёт дисбаланса
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

start = time.time()
XGB.fit(X_train, y_train)
print(f"{(time.time() - start)/60:.2f} minutes")

proba = XGB.predict_proba(X_test)[:, 1]  # вероятность положительного класса

auc = roc_auc_score(y_test, proba)

print(f"AUC = {auc:.4f}")

14.20 minutes
AUC = 0.8238


In [ ]:
LGBM = LGBMClassifier(
    n_estimators=300,
    max_depth=3,
    learning_rate=0.1,
    class_weight="balanced",       # встроенная поддержка дисбаланса
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

start = time.time()
LGBM.fit(X_train, y_train)
print(f"{(time.time() - start)/60:.2f} minutes")

proba = LGBM.predict_proba(X_test)[:, 1]  # вероятность положительного класса

auc = roc_auc_score(y_test, proba)

print(f"AUC = {auc:.4f}")

[LightGBM] [Info] Number of positive: 1307, number of negative: 2870434
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.140771 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1229
[LightGBM] [Info] Number of data points in the train set: 2871741, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

Обучаю на всех данных

In [ ]:
import time

clf = RandomForestClassifier(n_estimators=20, max_depth=5, n_jobs=1)
start = time.time()
clf.fit(X, y)
print(f"{(time.time() - start)/60:.2f} minutes")

0.96 minutes


# Проверка методов класса

In [ ]:
# model = WeatherRiskModel(model=model, feature_columns=X_train.columns.tolist(), api_key="008137874f864b7eba3220312252006")  # <- экземпляр класса

# route = [
#     (43.65, -79.38),
#     (43.66, -79.37),
#     (43.67, -79.35),
#     (43.68, -79.33)
# ]
# gh5 = model._coords_to_geohash5(route)
# print("geohash5 по маршруту:", gh5)


# weather_df = model._get_current_weather()
# print("Погода сейчас:")
# print(weather_df[['time', 'temp_c', 'humidity']])


# X_input = model._prepare_input(weather_df, gh5)
# print("Подготовленные данные:")
# print(X_input.head())
# print("Форма:", X_input.shape)


# risk = model.predict_sum_from_route(route)
# print("Суммарная вероятность ДТП:", risk)

# Сохраняю .pkl файл

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Toronto KSI/')
from model import ProbSumModel  # импортируем именно из model.py

In [ ]:
model = ProbSumModel(XGB)  # оборачиваем модель

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/Toronto KSI/model_xgb.pkl'
dump(model, path)

['/content/drive/MyDrive/Colab Notebooks/Toronto KSI/model_xgb.pkl']

# The END

In [ ]:
# обертка для суммы вероятностей по маршруту
class ProbSumModel:
    def __init__(self, base_model):
        self.model = base_model

    def predict_sum(self, X):
        probs = self.model.predict_proba(X)[:, 1]
        return probs.sum()         # returns the sum of P(class=1) over all rows in X

In [ ]:
wrapper = ProbSumModel(model)
path = '/content/drive/MyDrive/Colab Notebooks/Toronto KSI/toronto_ksi_risk.pkl'
dump(wrapper, path)

['/content/drive/MyDrive/Colab Notebooks/Toronto KSI/toronto_ksi_risk.pkl']

In [ ]:
# сохранение через класс

model = WeatherRiskModel(model=clf, feature_columns=X_train.columns.tolist(), api_key='008137874f864b7eba3220312252006')
dump(model, 'weather_model.pkl')

['weather_model.pkl']

In [ ]:
API_KEY  = "008137874f864b7eba3220312252006"
LOCATION = "Toronto"

tz = pytz.timezone("America/Toronto")
now = dt.datetime.now(tz).replace(minute=0, second=0, microsecond=0)
today = now.date()

# На всякий случай берём и вчера, если текущий час — 00:00
dates_needed = {today}
if now.hour == 0:
    dates_needed.add(today - dt.timedelta(days=1))

records = []

for date_ in dates_needed:
    url = (
        "https://api.weatherapi.com/v1/history.json"
        f"?key={API_KEY}&q={LOCATION}&dt={date_:%Y-%m-%d}&aqi=no&alerts=no"
    )
    resp = requests.get(url, timeout=30)
    resp.raise_for_status()
    data = resp.json()

    for hour in data["forecast"]["forecastday"][0]["hour"]:
        ts = dt.datetime.fromtimestamp(hour["time_epoch"], tz)
        if ts == now:
            records.append(hour)

full_current_weather = pd.json_normalize(records)

full_current_weather

,time_epoch,time,temp_c,temp_f,is_day,wind_mph,wind_kph,wind_degree,wind_dir,pressure_mb,pressure_in,precip_mm,precip_in,snow_cm,humidity,cloud,feelslike_c,feelslike_f,windchill_c,windchill_f,heatindex_c,heatindex_f,dewpoint_c,dewpoint_f,will_it_rain,chance_of_rain,will_it_snow,chance_of_snow,vis_km,vis_miles,gust_mph,gust_kph,uv,condition.text,condition.icon,condition.code
0,1751382000,2025-07-01 11:00,23.6,74.5,1,4.5,7.2,317,NW,1000.0,29.54,0.0,0.0,0.0,82,82,25.6,78.1,23.6,74.5,25.6,78.1,21.6,70.8,0,0,0,0,10.0,6.0,6.7,10.8,5.0,Cloudy,//cdn.weatherapi.com/weather/64x64/day/119.png,1006


In [ ]:
full_current_weather = full_current_weather[['time', 'temp_c', 'dewpoint_c', 'humidity', 'wind_kph', 'vis_km', 'pressure_mb']]

In [ ]:
full_current_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   time         1 non-null      object 
 1   temp_c       1 non-null      float64
 2   dewpoint_c   1 non-null      float64
 3   humidity     1 non-null      int64  
 4   wind_kph     1 non-null      float64
 5   vis_km       1 non-null      float64
 6   pressure_mb  1 non-null      float64
dtypes: float64(5), int64(1), object(1)
memory usage: 188.0+ bytes


In [ ]:
# Преобразуем колонку DATE в datetime, если это ещё не сделано
full_current_weather['time'] = pd.to_datetime(full_current_weather['time'])

# Извлекаем месяц и день
full_current_weather['Month'] = full_current_weather['time'].dt.month
full_current_weather['Day'] = full_current_weather['time'].dt.day
full_current_weather['Hour'] = full_current_weather['time'].dt.hour

full_current_weather.drop('time', axis=1, inplace=True)

/tmp/ipython-input-53-149136340.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_current_weather['time'] = pd.to_datetime(full_current_weather['time'])
/tmp/ipython-input-53-149136340.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_current_weather['Month'] = full_current_weather['time'].dt.month
/tmp/ipython-input-53-149136340.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [ ]:
full_current_weather

,temp_c,dewpoint_c,humidity,wind_kph,vis_km,pressure_mb,Month,Day,Hour
0,23.6,21.6,82,7.2,10.0,1000.0,7,1,11


In [ ]:
# Создаём DataFrame из geohash5
geo_hash_df = pd.DataFrame({'geohash5': all_geohash5})

# Картезианское произведение (все комбинации одной строки с каждым geohash5)
full_current_weather = full_current_weather.merge(geo_hash_df, how='cross')

In [ ]:
full_current_weather['Latitude'], full_current_weather['Longitude'] = zip(*full_current_weather['geohash5'].apply(pgh.decode))
full_current_weather.drop('geohash5', axis=1, inplace=True)

In [ ]:
full_current_weather = full_current_weather[X_train.columns]
full_current_weather

,Month,Day,temp_c,dewpoint_c,humidity,wind_kph,vis_km,pressure_mb,Hour,Latitude,Longitude
0,7,1,23.6,21.6,82,7.2,10.0,1000.0,11,43.571777,-79.650879
1,7,1,23.6,21.6,82,7.2,10.0,1000.0,11,43.571777,-79.606934
2,7,1,23.6,21.6,82,7.2,10.0,1000.0,11,43.571777,-79.562988
3,7,1,23.6,21.6,82,7.2,10.0,1000.0,11,43.571777,-79.519043
4,7,1,23.6,21.6,82,7.2,10.0,1000.0,11,43.571777,-79.475098
...,...,...,...,...,...,...,...,...,...,...,...
86,7,1,23.6,21.6,82,7.2,10.0,1000.0,11,43.835449,-79.211426
87,7,1,23.6,21.6,82,7.2,10.0,1000.0,11,43.791504,-79.167480
88,7,1,23.6,21.6,82,7.2,10.0,1000.0,11,43.791504,-79.123535
89,7,1,23.6,21.6,82,7.2,10.0,1000.0,11,43.835449,-79.167480


In [ ]:
import time

clf = RandomForestClassifier(n_estimators=20, max_depth=5, n_jobs=1)
start = time.time()
clf.fit(X_train, y_train)
print(f"{(time.time() - start)/60:.2f} minutes")

1.10 minutes


In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

Precision: 0.0
Recall: 0.0
F1 Score: 0.0


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, y_pred))

[[637865      0]
 [   300      0]]


In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9995299021412958


In [ ]:
clf.predict_proba(full_current_weather)[:,1]

array([1.42752197e-04, 1.45150670e-04, 1.77972792e-04, 1.74501017e-04,
       1.74501017e-04, 1.74501017e-04, 1.48807897e-04, 1.29606629e-04,
       1.29606629e-04, 1.29606629e-04, 7.63393282e-05, 5.16855513e-05,
       4.53544338e-05, 1.63664202e-04, 8.63726752e-04, 6.25124386e-04,
       5.63052100e-04, 1.66062674e-04, 2.23591970e-04, 8.77072303e-04,
       1.04828481e-03, 2.20120195e-04, 2.20120195e-04, 1.06885997e-03,
       1.09155286e-03, 6.38469937e-04, 8.09682444e-04, 5.76397651e-04,
       7.47610159e-04, 8.30257607e-04, 8.52950497e-04, 7.68185322e-04,
       7.90878211e-04, 4.77045450e-04, 4.41248379e-04, 4.90391002e-04,
       6.39508944e-04, 4.54593930e-04, 6.03711873e-04, 6.82178672e-04,
       7.04871561e-04, 6.46381600e-04, 6.69074490e-04, 2.20120195e-04,
       1.54798073e-04, 1.09155286e-03, 1.09155286e-03, 1.35596805e-04,
       1.35596805e-04, 1.09155286e-03, 1.09155286e-03, 8.52950497e-04,
       8.52950497e-04, 7.90878211e-04, 7.90878211e-04, 8.52950497e-04,
      

Function to get probabilities from the model

In [ ]:
class WeatherRiskModel:
    def __init__(self, model, feature_columns, api_key):
        self.model = model
        self.feature_columns = feature_columns
        self.api_key = api_key

    def _coords_to_geohash5(self, route_coords):
        return sorted({pgh.encode(lat, lon, precision=5) for lat, lon in route_coords})

    def _get_current_weather(self):
        tz = pytz.timezone("America/Toronto")
        now = dt.datetime.now(tz).replace(minute=0, second=0, microsecond=0)
        date_str = now.strftime("%Y-%m-%d")

        url = (
            f"https://api.weatherapi.com/v1/history.json"
            f"?key={self.api_key}&q=Toronto&dt={date_str}&aqi=no&alerts=no"
        )
        resp = requests.get(url, timeout=30)
        data = resp.json()

        # Выбираем только текущий час
        current_hour = [
            h for h in data["forecast"]["forecastday"][0]["hour"]
            if dt.datetime.fromtimestamp(h["time_epoch"], tz) == now
        ]
        return pd.json_normalize(current_hour)

    def _prepare_input(self, weather_df, geohash5_list):
        df = weather_df[['time', 'temp_c', 'dewpoint_c', 'humidity', 'wind_kph', 'vis_km', 'pressure_mb']].copy()
        df['time'] = pd.to_datetime(df['time'])
        df['Month'] = df['time'].dt.month
        df['Day'] = df['time'].dt.day
        df['Hour'] = df['time'].dt.hour
        df.drop('time', axis=1, inplace=True)

        geo_df = pd.DataFrame({'geohash5': geohash5_list})
        df = df.merge(geo_df, how='cross')
        df['Latitude'], df['Longitude'] = zip(*df['geohash5'].apply(pgh.decode))
        df.drop('geohash5', axis=1, inplace=True)

        return df[self.feature_columns]

    def predict_sum_from_route(self, route_coords):
        geohash5_list = self._coords_to_geohash5(route_coords)
        weather_df = self._get_current_weather()
        model_input = self._prepare_input(weather_df, geohash5_list)
        probs = self.model.predict_proba(model_input)[:, 1]
        return probs.sum()


Получить список geohash5, покрывающий маршрут

In [ ]:
def coords_to_geohash5(route_coords):
    geohashes = set()
    for lat, lon in route_coords:
        geohash = pgh.encode(lat, lon, precision=5)
        geohashes.add(geohash)
    return sorted(geohashes)